# Imports

In [1]:
import sys
sys.path.append('../')
from datetime import datetime
import pandas as pd

from run_all.main_preprocess import load_data, add_features
from utilities.utilities import get_latest_file

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Preprocess

In [ ]:
# %%time
# # Load original sources and combine to one DataFrame
# df_dataset_WMO = load_data()

In [ ]:
# %%time
# # Feature engineering to get more features
# df_dataset_WMO_with_features = add_features(df_dataset_WMO)

## Optional: Write temporary result

In [ ]:
# suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

# df_dataset_WMO_with_features.to_parquet(f'../../data/df_preprocess_WMO_{suffix_datetime}.parquet.gzip',
#               compression='gzip')

# Train

## Optional: Load previous dataset

In [2]:
## Continue with loaded data from preprocess
#df = df_dataset_WMO_with_features.copy()

# ## HARDCODED
# datapath = '../../data/'
# input_filename = 'df_preprocess_WMO_202103211137.parquet.gzip'
# df = pd.read_parquet(datapath + input_filename)

# ## SELECT LAST FILE
datapath = '../../data/'
df = get_latest_file(filename_str_contains='df_preprocess_', datapath=datapath, filetype='parquet')

## Train model

### Train imports

In [3]:
# zorgen voor de juiste modules
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV, cross_validate, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, BayesianRidge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor, XGBClassifier

from ColumnSelector import ColumnSelector

# instellingen voor panda weergave aanpassen
pd.set_option('display.max_rows', 500) # alle rijen tonen
pd.set_option('display.max_columns', 500) # alle kolommen tonen
pd.set_option('display.width', 1000) # kolombreedte
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.15f}'.format(x)) # floats output tot 15 decimalen

### Settings

In [4]:
## Dataframe parameters
# locatie van dataset 
DF_LOCATION = 'C:/_NoBackup/Git/__JADS/WMO_execute_group_project/data/df_dataset_WMO.parquet.gzip'
# Location all data
datapath = '../../data/'
# manier van laden dataset. Bijvoorbeeld read_parquet of read_csv
DF_READ = pd.read_parquet

## X & Y parameters
# de kolommen die uit de X dataset moeten worden gehaald. Dat is in ieder geval de y en eventueel nog meer kolommen.
# X_DROP_VALUES = ['wmoclienten', 'eenpersoonshuishoudens', 'huishoudenszonderkinderen', 'huishoudensmetkinderen']
X_DROP_VALUES = ['wmoclienten', 'percentagewmoclienten','wmoclientenper1000inwoners','perioden']
# de kolom die wordt gebruikt als y value
Y_VALUE = ['wmoclientenper1000inwoners']
# test size voor de train/test split
TEST_SIZE = 0.3
# random state voor de train/test split. Bijvoorbeeld random_state = 42 als vaste seed voor reproduceerbaarheid
RANDOM_STATE = 42

## Pipeline parameters
# strategy en waarde om te vullen bij lege categorische kolommen
NAN_VALUES_CAT_STRATEGY = 'constant'
NAN_VALUES_CAT_VALUES = 'Missing'
# waarden om in te vullen bij lege numerieke kolommen. Bijvoorbeeld mean of median
NAN_VALUES_NUM_STRATEGY = 'mean'
# 
#COLS_SELECT = ['aantalinwoners', 'mannen', 'vrouwen', 'k0tot15jaar'
#               , 'k15tot25jaar', 'k25tot45jaar', 'k45tot65jaar', 'k65jaarofouder', 'gescheiden'
#               , 'verweduwd', 'westerstotaal', 'sterftetotaal', 'gemiddeldehuishoudensgrootte'
#               , 'gemiddeldewoningwaarde', 'koopwoningen', 'huurwoningentotaal', 'inbezitwoningcorporatie'
#               , 'gemiddeldinkomenperinkomensontvanger', 'k40personenmetlaagsteinkomen', 'k20personenmethoogsteinkomen'
#               , 'actieven1575jaar', 'k40huishoudensmetlaagsteinkomen', 'k20huishoudensmethoogsteinkomen'
#               , 'huishoudensmeteenlaaginkomen', 'personenpersoortuitkeringaow', 'rucultuurrecreatieoverigediensten'
#               , 'personenautosperhuishouden', 'matevanstedelijkheid']
COLS_SELECT = None

## Model parameters

# manier van cross validate in de modellen. Bijvoorbeeld 10 of RepeatedKFold(n_splits=30, n_repeats=5, random_state=1)
CROSS_VALIDATE = 5
# manier van scoren in de modellen
MODEL_SCORING = 'neg_mean_squared_error'
## Grid Search parameters

# parameters die gebruikt worden in de grid search

ALPHA = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
NEIGHBORS = [3, 5, 11, 19]
NORMALIZE = [True, False]
KERNEL = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
GAMMA = [0.5, 1, 1.5, 2, 5]
N_ESTIMATORS = [50,100,200]
C_REGULARIZATION = [0.001, 0.01, 0,1, 1]

### Functions

In [ ]:
def drop_nan_from_specific_columns (df,columns_to_check):
    """
    Drops all rows with nan values in specific columns in a dataframe
    """
    df.dropna(
        axis=0,
        how='any',
        thresh=None,
        subset=columns_to_check,
        inplace=True
    )

In [ ]:
def split_clf_and_params(best_estimator_clf):
    """
    takes best estimator[clf] and outputs a list with clf and parameters
    """
    clf_and_params = str(best_estimator_clf)
    clf_and_params = clf_and_params.replace(")", "")
    clf_and_params_split = clf_and_params.split("(")
    return clf_and_params_split

In [ ]:
# functie maken om op basis van de grid search best estimator, het beste RMSE model te selecteren 
def rmse_from_gridsearch_best_estimator(grid_search):
    """
    Calculates RMSE from the grid search best estimator
    """
    rmse = np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test)))
    return (rmse)

In [ ]:
def rmse_from_neg_mean_squared_error(neg_mean_squared_error):
    """
    Calculates RMSE from the neq mean squared error
    """
    rmse = np.sqrt(-(neg_mean_squared_error))
    return (rmse)

### Load data

In [ ]:
# ## Done before start of 'Train' chapter
# df = get_latest_file(mypath=datapath)

#### Stappen hieronder mogelijk verplaatsten naar prepare stap, later beoordelen

In [ ]:
# checken of er rijen in het dataframe zitten waarbij de Y_value leeg is. Die rijen worden eruit gehaald.
drop_nan_from_specific_columns(df,Y_VALUE)

In [ ]:
# X en y aanmaken
X = df.drop(X_DROP_VALUES, axis=1)
y = df[Y_VALUE]
# splitsen van X en y in train/test. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)

In [ ]:
# splitsen van X_train in categorische en numerieke kolommen, om apart te kunnen transformeren
cat_cols = X_train.select_dtypes(include=['category']).columns
num_cols = X_train.select_dtypes(include=['int64','float64','float32','int32']).columns

### Pipelines

In [ ]:
# pipelines (pl) maken voor imputing, scaling en OneHotEncoding per datatype 

# categorie met waarde die is gegeven aan "MISSING" toevoegen
for col in cat_cols:
    # need to add category for missings, otherwise error with OneHotEncoding (volgens mij ook met alleen imputing)
    X_train[col].cat.add_categories(NAN_VALUES_CAT_VALUES, inplace=True)
categories = [X_train[col].cat.categories for col in cat_cols]

# pipeline voor categorial datatype
pl_ppc_cat = make_pipeline(
     SimpleImputer(
         missing_values = np.nan
        ,strategy = NAN_VALUES_CAT_STRATEGY
        ,fill_value = NAN_VALUES_CAT_VALUES)
    ,OneHotEncoder(categories=categories)
)

# pipeline voor numeriek datatype
pl_ppc_num = make_pipeline(
      ColumnSelector(cols=COLS_SELECT)
    ,SimpleImputer(
         missing_values = np.nan
        ,strategy = NAN_VALUES_NUM_STRATEGY)
    ,StandardScaler()
    #,PCA() # PCA heeft behoorlijk wat (positieve) invloed op de scores
)

In [ ]:
# pipelines maken om de preprocessing van de imputing te combineren
pl_ppc_total = make_column_transformer(
     (pl_ppc_cat, cat_cols)
    ,(pl_ppc_num, num_cols)
    ,remainder = 'drop'
)

## Gridsearch

In [ ]:
# pipeline maken om in de grid search te kunnen gebruiken
pl_gs_total = Pipeline([('preprocess', pl_ppc_total),
                       ('clf', LinearRegression())]) # Placeholder Estimator
    
# param grid waarin alle classifiers + hyper parameters kunnen worden opgenomen. 
# hier classifiers (modellen) + parameters toevoegen
param_grid_total = [{'clf': [LinearRegression()], 
                     'clf__normalize': NORMALIZE,},
                    
                    {'clf': [Ridge()],  
                     'clf__alpha': ALPHA},
                    
                    {'clf': [Lasso()], 
                     'clf__alpha': ALPHA},
                   
                 #   {'clf': [KNeighborsRegressor()],  
                 #    'clf__n_neighbors': NEIGHBORS},
                     
                 #  {'clf': [SVR()], 
                 #   'clf__kernel': KERNEL,
                 #   'clf__C': C_REGULARIZATION},
                    
                   {'clf': [XGBRegressor()],  
                    'clf__gamma': GAMMA,
                    'clf__n_estimators': N_ESTIMATORS},                   
                   ]
    
# grid search aanmaken
grid_search_total = GridSearchCV(pl_gs_total, param_grid_total, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)

In [ ]:
%%time
#grid search uitvoeren
grid_search_total.fit(X_train, y_train)

### Evaluate

In [ ]:
# de best estimator uit de grid search halen (beste train score)
print(f"Het model met de beste train score is:\n{grid_search_total.best_estimator_['clf']}")
# de RMSE berekenen voor de best estimator
print(f"Dit model heeft een train score RMSE van {rmse_from_neg_mean_squared_error(grid_search_total.best_score_)}") 
print(f"Dit model heeft een test score RMSE van  {rmse_from_gridsearch_best_estimator(grid_search_total)}")

### Save best model and best model properties

In [ ]:
# opslaan van beste estimator vanuit de gridsearch naar een Pickle file
suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')
output_filename = f'../../data/best_model_{suffix_datetime}.pickle'
pickle.dump(grid_search_total.best_estimator_, open(output_filename, 'wb'))

In [ ]:
# extra regel om tijdelijk een dummy bij input_filename te krijgen
input_filename = 'Hier komt uiteindelijk de input_filename_locatie'
# dictionary maken van alle properties die van het beste model moeten worden opgeslagen
best_model_properties_dict = {"Model": [split_clf_and_params(grid_search_total.best_estimator_['clf'])[0]],
                        "Gridsearch_Params": [split_clf_and_params(grid_search_total.best_estimator_['clf'])[1]],
                        "Train_RMSE": [rmse_from_neg_mean_squared_error(grid_search_total.best_score_)],
                        "Test_RMSE": [rmse_from_gridsearch_best_estimator(grid_search_total)],
                        "Number_of_features": [len(X.columns)],
                        "Y_value": Y_VALUE,
                        "Input_filename": [input_filename],
                        "Output_filename": [output_filename],
                                     }
best_model_properties = pd.DataFrame(best_model_properties_dict)

In [ ]:
best_model_properties

In [ ]:
# opslaan van beste model properties naar csv
best_model_properties.to_csv(f'../../data/best_model_properties{suffix_datetime}.csv', index = False, header=True)

In [5]:
# combineren van de verschillende best model properties csv's naar één dataframe
import os
import glob
os.chdir(datapath)

In [6]:
# combineren van de verschillende best model properties csv's naar één dataframe
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_df = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_df

,Model,Gridsearch_Params,Train_RMSE,Test_RMSE,Number_of_features,Y_value,Input_filename,Output_filename
0,Ridge,alpha=1,0.000875748016430,0.000874619538451,140,wmoclientenper1000inwoners,Hier komt uiteindelijk de input_filename_locatie,../../data/best_model_202104021328.pickle
0,Ridge,alpha=1,0.000875748016430,0.000874619538451,140,wmoclientenper1000inwoners,Hier komt uiteindelijk de input_filename_locatie,../../data/best_model_202104021400.pickle


## Code voor testen pickle file & predict

In [ ]:
# # pickle file inladen voor predict
# loaded_model = get_latest_file(output_filename_str_contains='best_model_', datapath=datapath, filetype='pickle')
# # hoe moet ik deze score interpreteren?
# result = loaded_model.score(X_test, y_test)
# print(result)
# loaded_model.predict(X_test)

In [ ]:
# regel om te testen of opgeslagen pickle file overeen komt met model
#grid_search_total.best_estimator_.predict(X_test)

In [ ]:
# dit is het beste model uit de grid search
#grid_search_total.best_estimator_